In [1]:
from z3 import *
import numpy as np
import re
import arviz as az
import pandas as pd
import itertools
import random
import operator # TODO: check this library
import matplotlib.pyplot as plt
plt.rcParams["figure.autolayout"] = True # to for tight_layout()

import os, sys
sys.path.append(os.path.join("../../../../"))

from src.mcmc_sat import smt, mcmc, sat, utils

In [2]:
#32 Haplotypes
#5 placings in the haplotype (2⁵ = 32)
#100 genotypes

haplotypes = list(map(list, itertools.product([0, 1], repeat=5)))
haplotypesProb = [
    0.0056,   0.004,                0.00368,  0.1,                 
    0.005681, 0.09,                 0.00468,  0.003,    
    0.08,     0.006147,             0.00595,  0.0071,   
    0.00345,  0.00632,              0.1,      0.0029,   
    0.0071,   0.006147,             0.085,    0.0045671,
    0.003379, 0.1,                  0.00461,  0.00614,             
    0.075,    0.08,                 0.00761,  0.004321, 
    0.09,     0.009217900000000112, 0.0134,   0.075]

haplotypes_prob_synth = [
    0.00100000000000000, 0.00100000000000000, 0.00100000000000000, 0.00100000000000000,
    0.00100000000000000, 0.32366666666666666, 0.00100000000000000, 0.00100000000000000,
    0.00100000000000000, 0.00100000000000000, 0.00100000000000000, 0.00100000000000000,   
    0.00100000000000000, 0.00100000000000000, 0.00100000000000000, 0.00100000000000000,
    0.00100000000000000, 0.00100000000000000, 0.32366666666666666, 0.00100000000000000,
    0.00100000000000000, 0.00100000000000000, 0.00100000000000000, 0.00100000000000000,
    0.00100000000000000, 0.00100000000000000, 0.00100000000000000, 0.00100000000000000,
    0.00100000000000000, 0.00100000000000000, 0.00100000000000000, 0.32366666666666666
]

In [3]:
# NOTE: We try a simpler version of the problem with all probability
# mass concentrated in 3 haplotypes
haplotypesProb=haplotypes_prob_synth

In [4]:
# NOTE: This is a dict to store the number of haplotypes drawn from the probabilities above
haplotypesnumber = {}
for i in range(32):
    haplotypesnumber[str(haplotypes[i])] = 0

In [5]:
# NOTE: This code snippet samples 100 genotypes by sampling haplotypes with the probabilities above
#       I suppose the goal is to recover the probabilities above
genotypes = []
d = {'h_1': [], 'h_2': [], 'g': []} # NOTE: added by Raúl
# seeds not working...
np.random.seed(2023)
random.seed(2023)
for i in range(100):
    h1 = np.random.choice(np.arange(32), p=haplotypesProb) # samples one haplotype (from 0 to 31) with probabilities above
    h2 = np.random.choice(np.arange(32), p=haplotypesProb) # samples one haplotype (from 0 to 31) with probabilities above
    genotypes.append(list(map(operator.add,haplotypes[h1],haplotypes[h2]))) # combines them to form a genotype
    d['h_1'].append(haplotypes[h1]); d['h_2'].append(haplotypes[h2]); d['g'].append(genotypes[i]); # NOTE: added by Raúl
    haplotypesnumber[str(haplotypes[h1])] = haplotypesnumber[str(haplotypes[h1])] +1 # keeps count of sampled haplotypes 
    haplotypesnumber[str(haplotypes[h2])] = haplotypesnumber[str(haplotypes[h2])] +1 # keeps count of sampled haplotypes 
df = pd.DataFrame(data=d)

In [6]:
df

,h_1,h_2,g
0,"[0, 0, 1, 0, 1]","[1, 1, 1, 1, 1]","[1, 1, 2, 1, 2]"
1,"[1, 0, 0, 1, 0]","[0, 0, 1, 0, 1]","[1, 0, 1, 1, 1]"
2,"[0, 0, 1, 0, 1]","[1, 0, 0, 1, 0]","[1, 0, 1, 1, 1]"
3,"[0, 0, 1, 0, 1]","[1, 1, 1, 1, 1]","[1, 1, 2, 1, 2]"
4,"[1, 0, 0, 1, 0]","[1, 0, 0, 1, 0]","[2, 0, 0, 2, 0]"
...,...,...,...
95,"[0, 0, 1, 0, 1]","[0, 0, 1, 0, 1]","[0, 0, 2, 0, 2]"
96,"[1, 1, 1, 1, 1]","[1, 1, 1, 1, 1]","[2, 2, 2, 2, 2]"
97,"[1, 1, 1, 1, 1]","[1, 1, 1, 1, 1]","[2, 2, 2, 2, 2]"
98,"[1, 0, 0, 1, 0]","[1, 1, 1, 1, 1]","[2, 1, 1, 2, 1]"


In [7]:
haplotypesnumber

{'[0, 0, 0, 0, 0]': 0,
 '[0, 0, 0, 0, 1]': 0,
 '[0, 0, 0, 1, 0]': 0,
 '[0, 0, 0, 1, 1]': 0,
 '[0, 0, 1, 0, 0]': 0,
 '[0, 0, 1, 0, 1]': 57,
 '[0, 0, 1, 1, 0]': 0,
 '[0, 0, 1, 1, 1]': 0,
 '[0, 1, 0, 0, 0]': 1,
 '[0, 1, 0, 0, 1]': 1,
 '[0, 1, 0, 1, 0]': 0,
 '[0, 1, 0, 1, 1]': 0,
 '[0, 1, 1, 0, 0]': 0,
 '[0, 1, 1, 0, 1]': 0,
 '[0, 1, 1, 1, 0]': 0,
 '[0, 1, 1, 1, 1]': 2,
 '[1, 0, 0, 0, 0]': 1,
 '[1, 0, 0, 0, 1]': 0,
 '[1, 0, 0, 1, 0]': 71,
 '[1, 0, 0, 1, 1]': 0,
 '[1, 0, 1, 0, 0]': 0,
 '[1, 0, 1, 0, 1]': 0,
 '[1, 0, 1, 1, 0]': 0,
 '[1, 0, 1, 1, 1]': 0,
 '[1, 1, 0, 0, 0]': 0,
 '[1, 1, 0, 0, 1]': 0,
 '[1, 1, 0, 1, 0]': 1,
 '[1, 1, 0, 1, 1]': 1,
 '[1, 1, 1, 0, 0]': 0,
 '[1, 1, 1, 0, 1]': 0,
 '[1, 1, 1, 1, 0]': 1,
 '[1, 1, 1, 1, 1]': 64}

In [8]:
# Assumption, all bit-vectors are of the same size
num_bits = 2
num_vars = 5*100*2
var_list = [BitVec(f'x{i}',num_bits) for i in range(num_vars)]
x = var_list
g = Goal()
sat.add_bool_vars_to_goal(g, var_list)

#Add contraints
for i in range(num_vars):
    g.add(ULE(x[i],1)) # Each BitVec is in {0,1}

for i in range(100): # iterate over all genotypes
    for j in range(5): # iterate over all bits of the BitVec
        g.add(x[i*10+j] + x[i*10+j+5] == genotypes[i][j])

In [9]:
trace = mcmc.sample_mh_trace_from_z3_model(backend='spur',
                                           z3_problem=g,
                                           num_vars=num_vars,
                                           num_bits=num_bits,
                                           num_samples=30000,
                                           num_chains=4)

Using default filename: "spur_input/samples_z3_problem.txt"
Performing Uniform Model Sampling...
Input File:  spur_input/z3_problem.cnf
Output File: spur_input/samples_z3_problem.txt

Preprocessing ... DONE
variables (all/used/free): 	4523/4523/0
independent support size:  	0
clauses (all/long/binary/unit): 9970/4092/4570/1308
Initialize cache
Size of Cacheable Component:	56
Size of MPZ Class:	16
Maximum cache size:	100617 MB

STAGE #1: Build the initial partial assignments

time elapsed: 61.2029s

time elapsed: 121.488s

time elapsed: 182.765s
STAGE #1: COMPLETED building initial partial assignments
STAGE #2 - Filling in partial assignments...
Initialize cache
Size of Cacheable Component:	56
Size of MPZ Class:	16
Maximum cache size:	100617 MB

Sample #1 of 1 is already a complete assignment.  Continuing...
STAGE #2 - COMPLETE


variables (total / active / free)	4523/4523/0
clauses (removed) 		9970 (0)
decisions 				215
conflicts 				0
conflict clauses (all/bin/unit) 	0/0/1308
failed l


time elapsed: 60.8148s



time elapsed: 121.708s


STAGE #1: COMPLETED building initial partial assignments
STAGE #2 - Filling in partial assignments...
Initialize cache
Size of Cacheable Component:	56
Size of MPZ Class:	16

 Free RAM 263MB
Maximum cache size:	691 MB

Sample #1 of 1 is already a complete assignment.  Continuing...
STAGE #2 - COMPLETE




variables (total / active / free)	4523/4523/0
clauses (removed) 		9970 (0)
decisions 				215
conflicts 				0
conflict clauses (all/bin/unit) 	0/0/1308
failed literals found by implicit BCP 	 0
implicit BCP miss rate 	0%
bytes cache size     	20253436	
bytes cache (overall) 	20253436
bytes cache (infra / comps) 20234408/19028	
bytes pure comp data (curr)    5156
bytes pure comp data (overall) 5156
bytes cache with sysoverh (curr)    32920
bytes cache with sysoverh (overall) 32920
cache (stores / hits) 			217/0
cache miss rate 		100%
avg. variable count (stores / hits) 	46.6406/0


# solutions 
52656145834278593348959013841835216159447547700274555627155488768

# END

time: 154.14s


Total Sampler Execution Time: 154.14s

#START_HEADER
start_time,Sun Feb 25 15:21:31 2024
formula_file,spur_input/z3_problem.cnf
num_vars,4523
independent_support_size,0
num_clauses,9970
tot_num_models,52656145834278593348959013841835216159447547700274555627155488768
max_component_split_depth,1
max_branch_va

In [10]:
var_names= ['x0','x1','x2','x3','x4']
trace.posterior[var_names].sel(chain=3,draw=range(30000-10,30000))

<xarray.Dataset> Size: 488B
Dimensions:  (draw: 10)
Coordinates:
    chain    int64 8B 3
  * draw     (draw) int64 80B 29990 29991 29992 29993 ... 29997 29998 29999
Data variables:
    x0       (draw) int64 80B 1 1 0 1 0 0 0 1 0 1
    x1       (draw) int64 80B 1 0 0 0 1 0 1 1 0 1
    x2       (draw) int64 80B 1 1 1 1 1 1 1 1 1 1
    x3       (draw) int64 80B 1 1 0 0 0 1 0 0 0 1
    x4       (draw) int64 80B 1 1 1 1 1 1 1 1 1 1
Attributes:
    created_at:     2025-03-22T17:45:18.591502+00:00
    arviz_version:  0.21.0

In [11]:
haplo_var_vectors = [[f'x{i+j}' for j in range(5) ] for i in np.arange(0,num_vars,step=5)]
haplo_var_vectors

[['x0', 'x1', 'x2', 'x3', 'x4'],
 ['x5', 'x6', 'x7', 'x8', 'x9'],
 ['x10', 'x11', 'x12', 'x13', 'x14'],
 ['x15', 'x16', 'x17', 'x18', 'x19'],
 ['x20', 'x21', 'x22', 'x23', 'x24'],
 ['x25', 'x26', 'x27', 'x28', 'x29'],
 ['x30', 'x31', 'x32', 'x33', 'x34'],
 ['x35', 'x36', 'x37', 'x38', 'x39'],
 ['x40', 'x41', 'x42', 'x43', 'x44'],
 ['x45', 'x46', 'x47', 'x48', 'x49'],
 ['x50', 'x51', 'x52', 'x53', 'x54'],
 ['x55', 'x56', 'x57', 'x58', 'x59'],
 ['x60', 'x61', 'x62', 'x63', 'x64'],
 ['x65', 'x66', 'x67', 'x68', 'x69'],
 ['x70', 'x71', 'x72', 'x73', 'x74'],
 ['x75', 'x76', 'x77', 'x78', 'x79'],
 ['x80', 'x81', 'x82', 'x83', 'x84'],
 ['x85', 'x86', 'x87', 'x88', 'x89'],
 ['x90', 'x91', 'x92', 'x93', 'x94'],
 ['x95', 'x96', 'x97', 'x98', 'x99'],
 ['x100', 'x101', 'x102', 'x103', 'x104'],
 ['x105', 'x106', 'x107', 'x108', 'x109'],
 ['x110', 'x111', 'x112', 'x113', 'x114'],
 ['x115', 'x116', 'x117', 'x118', 'x119'],
 ['x120', 'x121', 'x122', 'x123', 'x124'],
 ['x125', 'x126', 'x127', 'x128', '

In [12]:
def __get_distinct_haplotype_counts(trace, var_names: list[str]):
    """
    Returns the count of each distinct sampled haplotype for the Bitvector specified by the variables in `var_names`
    Note: Try to reimplement this function more efficiently. The bottleneck seem to be the casting of integer lists to str
    """
    return np.unique(np.array([str(a).replace(" ", ", ") for a in trace.posterior[var_names].to_array().values.reshape(5,-1).T]), return_counts=True)

def get_count_haplotypes(trace, num_vars, haplotypes):
    """
    Returns a dictionary where keys are haplotypes and values are the counts of each haplotype in the trace.
    Note: It is computationally expensive due to the function `__get_distinct_haplotype_counts`.
    """
    # list of lists where each list contains the variables for each BitVector (to be used as indexes in the posterior trace)
    haplo_var_vectors = [[f'x{i+j}' for j in range(5) ] for i in np.arange(0,num_vars,step=5)]
    # list of pairs where the first element in each pair is list with
    # each distinct haplotype, and the second pair element is a list
    # with the counts of the corresponding haplotype
    counts_per_var_vector = [__get_distinct_haplotype_counts(trace, haplo_var_vector) for haplo_var_vector in haplo_var_vectors]
    # initilize a dict with distinct haplotypes as keys and values are counts of each haplotype (initially 0)
    haplotypesnumber_posterior = { str(haplotypes[i]): 0 for i in range(32) }
    # iterate over all haplotype counts for each BitVector, and add them to the dictionary above
    for i in counts_per_var_vector:
        num_distinct_haplotypes = len(i[0])
        for j in range(num_distinct_haplotypes):
            haplotypesnumber_posterior[i[0][j]] += i[1][j]
    # return dictionary with all counts
    return haplotypesnumber_posterior

In [ ]:
haplotypesnumber_posterior = get_count_haplotypes(trace, num_vars, haplotypes)

In [ ]:
## Data from previous run

# haplotypesnumber_posterior = {
#  '[0, 0, 0, 0, 0]': 247004,
#  '[0, 0, 0, 0, 1]': 607128,
#  '[0, 0, 0, 1, 0]': 307620,
#  '[0, 0, 0, 1, 1]': 818288,
#  '[0, 0, 1, 0, 0]': 412276,
#  '[0, 0, 1, 0, 1]': 1103424,
#  '[0, 0, 1, 1, 0]': 472740,
#  '[0, 0, 1, 1, 1]': 713544,
#  '[0, 1, 0, 0, 0]': 653180,
#  '[0, 1, 0, 0, 1]': 381460,
#  '[0, 1, 0, 1, 0]': 594260,
#  '[0, 1, 0, 1, 1]': 411860,
#  '[0, 1, 1, 0, 0]': 878104,
#  '[0, 1, 1, 0, 1]': 577520,
#  '[0, 1, 1, 1, 0]': 1294472,
#  '[0, 1, 1, 1, 1]': 727120,
#  '[1, 0, 0, 0, 0]': 759108,
#  '[1, 0, 0, 0, 1]': 578796,
#  '[1, 0, 0, 1, 0]': 1208096,
#  '[1, 0, 0, 1, 1]': 335244,
#  '[1, 0, 1, 0, 0]': 619504,
#  '[1, 0, 1, 0, 1]': 1373360,
#  '[1, 0, 1, 1, 0]': 411388,
#  '[1, 0, 1, 1, 1]': 352480,
#  '[1, 1, 0, 0, 0]': 1760908,
#  '[1, 1, 0, 0, 1]': 1373004,
#  '[1, 1, 0, 1, 0]': 531760,
#  '[1, 1, 0, 1, 1]': 472284,
#  '[1, 1, 1, 0, 0]': 1449520,
#  '[1, 1, 1, 0, 1]': 905704,
#  '[1, 1, 1, 1, 0]': 760060,
#  '[1, 1, 1, 1, 1]': 908784
#}

In [ ]:
def normalize_haplotypes_posterior(haplo_num_posterior):
    """
    Takes as input a dict with haplotypes as keys and their counts and values.
    Returns a similar dict where values is the probability of each haplotype (frequentist)
    """
    total = sum([haplo_num_posterior[x] for x in list(haplo_num_posterior.keys())])
    return { x: haplo_num_posterior[x]/total  for x in list(haplo_num_posterior.keys())}

In [ ]:
c = normalize_haplotypes_posterior(haplotypesnumber_posterior)
d = normalize_haplotypes_posterior(haplotypesnumber)

In [ ]:
posterior_probs = [c[k] for k in c.keys()]
posterior_probs_sample = [d[k] for k in d.keys()]

In [ ]:
df_p = pd.DataFrame(data={'MCMC+SAT': posterior_probs, 'population': haplotypesProb, 'sample': posterior_probs_sample})

In [ ]:
df_p

In [ ]:
df_p.plot.bar(figsize=(15,4), color=['blue', 'fuchsia', 'darkorange'], edgecolor='black');
plt.savefig('haplo_mcmc_sat.png',dpi=150);

In [ ]:
# comparing haplotype probabilities with original
fig, ax = plt.subplots(1,figsize=(15,4))
ax.bar(range(32), [abs(posterior_probs[i] - haplotypesProb[i]) for i in range(len(haplotypesProb))]);

In [ ]:
[abs(posterior_probs[i] - haplotypesProb[i]) for i in range(len(haplotypesProb))]

An attempt to use the posterior distribution to draw conclusions

In [ ]:
plot_posterior_haplotype = lambda var_names, ax: az.plot_dist(np.array(list(map(lambda x: int("".join(str(y) for y in x),2),trace.posterior[var_names].to_array().values.reshape(5,-1).T))),ax=ax)

num_vars = 200
num_haplotype_vars = int(num_vars/5)
num_rows = int(num_haplotype_vars/4)
size_y   = 3*num_rows

fig, axs_m = plt.subplots(nrows=num_rows, ncols=4, figsize=(23,size_y))
axs = axs_m.ravel()
for i in np.arange(0,num_vars,step=5):
    var_names = [f'x{i+j}' for j in range(5)]
    ax = axs[int(i/5)]
    plot_posterior_haplotype(var_names, ax=ax)
    ax.set_title(var_names)
# plt.tight_layout();
plt.savefig('posterior_haplotypes.png',dpi=150);
plt.show();

In [ ]:
# Plot the posterior
az.plot_trace(trace, var_names=['x0', 'x1', 'x2', 'x3', 'x4']);
plt.savefig('spur_trace_plot_haplotypes.png', dpi=150);

In [ ]:
# Diagnostics
az.summary(trace)

/home/pardo/programs/miniconda3/envs/mcmc+sat/lib/python3.11/site-packages/arviz/stats/diagnostics.py:592: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


/home/pardo/programs/miniconda3/envs/mcmc+sat/lib/python3.11/site-packages/arviz/stats/diagnostics.py:592: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


/home/pardo/programs/miniconda3/envs/mcmc+sat/lib/python3.11/site-packages/arviz/stats/diagnostics.py:592: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


/home/pardo/programs/miniconda3/envs/mcmc+sat/lib/python3.11/site-packages/arviz/stats/diagnostics.py:592: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
/home/pardo/programs/miniconda3/envs/mcmc+sat/lib/python3.11/site-packages/arviz/stats/diagnostics.py:592: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
/home/pardo/programs/miniconda3/envs/mcmc+sat/lib/python3.11/site-packages/arviz/stats/diagnostics.py:592: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


/home/pardo/programs/miniconda3/envs/mcmc+sat/lib/python3.11/site-packages/arviz/stats/diagnostics.py:592: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
/home/pardo/programs/miniconda3/envs/mcmc+sat/lib/python3.11/site-packages/arviz/stats/diagnostics.py:592: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
/home/pardo/programs/miniconda3/envs/mcmc+sat/lib/python3.11/site-packages/arviz/stats/diagnostics.py:592: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


...

/home/pardo/programs/miniconda3/envs/mcmc+sat/lib/python3.11/site-packages/arviz/stats/diagnostics.py:592: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
/home/pardo/programs/miniconda3/envs/mcmc+sat/lib/python3.11/site-packages/arviz/stats/diagnostics.py:592: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


/home/pardo/programs/miniconda3/envs/mcmc+sat/lib/python3.11/site-packages/arviz/stats/diagnostics.py:592: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
/home/pardo/programs/miniconda3/envs/mcmc+sat/lib/python3.11/site-packages/arviz/stats/diagnostics.py:592: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


/home/pardo/programs/miniconda3/envs/mcmc+sat/lib/python3.11/site-packages/arviz/stats/diagnostics.py:592: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
/home/pardo/programs/miniconda3/envs/mcmc+sat/lib/python3.11/site-packages/arviz/stats/diagnostics.py:592: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


/home/pardo/programs/miniconda3/envs/mcmc+sat/lib/python3.11/site-packages/arviz/stats/diagnostics.py:592: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
x0,0.502,0.5,0.0,1.0,0.001,0.001,119977.0,119977.0,1.0
x1,0.499,0.5,0.0,1.0,0.001,0.001,120855.0,120000.0,1.0
x2,1.000,0.0,1.0,1.0,0.000,0.000,120000.0,120000.0,NaN
x3,0.498,0.5,0.0,1.0,0.001,0.001,119947.0,119947.0,1.0
x4,1.000,0.0,1.0,1.0,0.000,0.000,120000.0,120000.0,NaN
...,...,...,...,...,...,...,...,...,...
x995,1.000,0.0,1.0,1.0,0.000,0.000,120000.0,120000.0,NaN
x996,0.497,0.5,0.0,1.0,0.001,0.001,120955.0,120000.0,1.0
x997,0.494,0.5,0.0,1.0,0.001,0.001,119539.0,119539.0,1.0
x998,1.000,0.0,1.0,1.0,0.000,0.000,120000.0,120000.0,NaN


In [ ]:
az.to_netcdf(trace, f'haplotypes_SAT_{num_vars}_{num_bits}.nc')